In [55]:
import pandas as pd, numpy as np
import sklearn
import scipy.sparse
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from bs4 import BeautifulSoup

In [80]:
data = pd.read_csv('../data/raw_data/kaggle/train.csv')
labels=(data["toxic"] | data["severe_toxic"] | data["obscene"] | data["threat"] | data["insult"] | data["identity_hate"])
# labels=data["Labels"]
train_data = data[0:100000]
test_data = data[100001:]
train_labels = labels[0:100000]
test_labels = labels[100001:]

corpus = data["comment_text"]

In [88]:
def review_to_words( corpus ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    new_corp = []
    i=0
    for rev in corpus:
        i+=1
        if(i%10000 == 0):
            print("iter", i/10000,"of", len(corpus)/10000)
        
        # 1. Remove HTML
        review_text = BeautifulSoup(rev).get_text() 
        #
        # 2. Remove non-letters        
        letters_only = re.sub("[^a-zA-Z']", " ", review_text)

        p = letters_only.split()
        new_corp.append(" ".join(p))
    return(new_corp)   

## Multinomial naive bayes

In [89]:
corpus = list(corpus)
new_corp=review_to_words(corpus)


iter 1.0 of 15.9571
iter 2.0 of 15.9571
iter 3.0 of 15.9571
iter 4.0 of 15.9571
iter 5.0 of 15.9571
iter 6.0 of 15.9571
iter 7.0 of 15.9571
iter 8.0 of 15.9571
iter 9.0 of 15.9571
iter 10.0 of 15.9571
iter 11.0 of 15.9571
iter 12.0 of 15.9571
iter 13.0 of 15.9571
iter 14.0 of 15.9571
iter 15.0 of 15.9571


In [102]:
def cross_val_count_vec(vectorizer, corpus):
    X = vectorizer.fit_transform(corpus)
    vocab = vectorizer.get_feature_names()
    train_feature_vectors_sparse = X[0:100000]
    test_feature_vectors_sparse = X[100001:]
    # TRAINING

    clf = MultinomialNB()
    clf.fit(train_feature_vectors_sparse, train_labels)

    # PREDICTION
    predictions = clf.predict(test_feature_vectors_sparse)


    # PERFORMANCE METRICS

    confmat = confusion_matrix(test_labels, predictions)

    precision = confmat[1,1]/(confmat[1,1] + confmat[1,0])
    recall = confmat[1,1]/(confmat[1,1] + confmat[0,1])
    fScore = (2*precision*recall)/(recall+precision)
    ccr = (confmat[0,0] + confmat[1,1])/(sum(sum(confmat)))

    print("Precision = ",precision)
    print("Recall = ",recall)
    print("F_score = ",fScore)
    print("CCR = ",ccr)
    print()
    print("Confusion Matrix: \n", confmat)
    
    return fScore

In [113]:
fsc = []
for i in range (1,6):
    
    vectorizer = CountVectorizer(strip_accents='unicode',lowercase=True,stop_words='english',ngram_range=(1, i))

    fsc.append(cross_val_count_vec(vectorizer, new_corp))
    

KeyboardInterrupt: 

In [110]:
vectorizer.get_feature_names()

[' ',
 "'",
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [104]:
vectorizer = CountVectorizer(strip_accents='unicode',lowercase=True,stop_words='english', analyzer="word",token_pattern=r'\w{1,}')

cross_val_count_vec(vectorizer, corpus)

Precision =  0.602088167053
Recall =  0.763716628127
F_score =  0.673338893522
CCR =  0.940825919087

Confusion Matrix: 
 [[52412  1124]
 [ 2401  3633]]
